In [1]:
#Import Dependencies
import pandas as pd
import time
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests

In [2]:
executable_path = {'executable_path': 'c:/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
#Mars news site scrape
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

#retrieve most recent news title and paragraph
news_title = soup.find('div', class_='content_title').text
news_p = soup.find('div', class_='article_teaser_body').text

print(news_title)
print(news_p)

Mars Now
Did you know NASA's next Mars rover has a nearly identical sibling on Earth for testing? Even better, it's about to roll for the first time through a replica Martian landscape.


In [4]:
#JPL Mars Image Connection
executable_path = {'executable_path': 'c:/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [7]:
#Mars image scrape
jpl_url = 'https://www.jpl.nasa.gov'
image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_url)
html = browser.html
image_soup = bs(html, 'html.parser')

In [14]:
#Feature Image Link
path = image_soup.find_all('img')[3]['src']
featured_image_url = jpl_url + path
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA24088-640x350.jpg


In [18]:
#Mars Facts
facts_url = 'https://space-facts.com/mars/'
#get tables
facts_df = pd.read_html(facts_url)[0]
#convert to an html table
facts_html = facts_df.to_html(index=False, header=False)

In [24]:
#Mars Hemispheres
spheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(spheres_url)
spheres_html = browser.html
spheres_soup = bs(spheres_html, 'html.parser')


In [29]:
# Mars hemispheres products data
all_mars_hemispheres = spheres_soup.find('div', class_='collapsible results')
mars_hemispheres = all_mars_hemispheres.find_all('div', class_='item')

hemisphere_image_urls = []

# Iterate through each hemisphere data
for i in mars_hemispheres:
    # Collect Title
    hemisphere = i.find('div', class_="description")
    title = hemisphere.h3.text
    
    # Collect image link by browsing to hemisphere page
    hemisphere_link = hemisphere.a["href"]    
    browser.visit(sphere_url + hemisphere_link)
    
    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    # Create Dictionary to store title and url info
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = image_url
    
    hemisphere_image_urls.append(image_dict)

print(hemisphere_image_urls)

NameError: name 'sphere_url' is not defined

In [25]:
# Mars hemispheres products data
all_spheres = spheres_soup.find('div', class_='collapsible results')
mars_spheres = all_spheres.find_all('div', class_='item')

sphere_image_urls = []

Iterate through each hemisphere data
for sphere in mars_spheres:
    # Collect Title
    hemisphere = sphere.find('div', class_="description")
    title = hemisphere.h3.text
    
    # Collect image link by browsing to hemisphere page
    hemisphere_link = hemisphere.a["href"]    
    browser.visit(spheres_url + hemisphere_link)
    
    image_html = browser.html
    image_soup = bs(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    # Create Dictionary to store title and url info
    image_dict = {}
    image_dict['title'] = title
    image_dict['img_url'] = image_url
    
    hemisphere_image_urls.append(image_dict)

print(hemisphere_image_urls)

AttributeError: 'NoneType' object has no attribute 'find'